In [76]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import sys
import re
import vaderSentiment as vs
import plotly.graph_objs as go
!{sys.executable} -m pip install textblob
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [77]:
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

def normalizedMean(numbers):
    return float(float(sum(numbers)) / max(len(numbers), 1))/max(len(numbers), 1)

In [78]:
def getNutrientRelevantStatementsPolarityListUsingTextBlob(year, nutrientName): 
    with open(year + '.txt', 'r') as myfile:
        text = myfile.read()
    text1 =text.replace('\n________________\n','\n')
    text2 = re.sub(r'\n([a-z]+)',r' \1', text1) # To remove erroneous new line characters '\n' in between a sentence.
    text3 =re.sub(r'\. ([a-z]+)',r' \1', text2) # To remove the last '.' used for abbreviations so that next steps dont confuse this with a new statement indicating '.'            
    list1 = [sentence for sentence in text3.split('\n')]
    list2 = []
    for i in list1:
        list2.append(i.split('.'))
    
    finalList = []
    
    for i in list2:
            for j in i:
                finalList.append(j)
    
    nutrientStatementList = []
    
    for i in finalList:
        if nutrientName.lower() in i:
            nutrientStatementList.append(i)
    
    
    statementsPolarityList = []
    
    for i in nutrientStatementList:
            statementsPolarityList.append(TextBlob(i).sentiment.polarity)
        
        
    return statementsPolarityList; 




In [79]:
def getPositivePolarityList(nutrientPolarityList): 
    positivePolarityList = []
    for i in nutrientPolarityList:
        if i>0:
               positivePolarityList.append(i)
    return positivePolarityList;  

def getNegativePolarityList(nutrientPolarityList): 
    negativePolarityList = []
    for i in nutrientPolarityList:
        if i<0:
               negativePolarityList.append(i)
    return negativePolarityList; 

def getNeutralPolarityList(nutrientPolarityList): 
    neutralPolarityList = []
    for i in nutrientPolarityList:
        if i==0:
               neutralPolarityList.append(i)
    return neutralPolarityList; 

In [80]:
def plotPolarityScatterPlot (nutrient,nutrientStatementPolarityList,year,libraryName):
    import plotly
    plotly.tools.set_credentials_file(username='shivam.saith', api_key='xCijI2Ae8pZolWXkXMxN')
    import plotly.plotly as py
    import plotly.graph_objs as go

    import numpy as np


    trace0 = go.Scatter(
        x = np.arange(1,1000,2),
        y = getPositivePolarityList(nutrientStatementPolarityList),
        name = 'Positive',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'rgba(0, 153, 51, .8)',
            line = dict(
                width = 2,
                color = 'rgb(0, 0, 0)'
            )
        )
    )

    trace1 = go.Scatter(
        x = np.arange(2,1001,2),
        y = getNegativePolarityList(nutrientStatementPolarityList),
        name = 'Negative',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'rgb(230, 46, 0)',
            line = dict(
                width = 2,
                color = 'rgb(0, 0, 0)'
            )
        )
    )

    trace2 = go.Scatter(
        x = np.arange(3,1002,3),
        y = getNeutralPolarityList(nutrientStatementPolarityList),
        name = 'Neutral',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'rgba(179, 179, 204, .8)',
            line = dict(
                width = 2,
                color = 'rgb(0, 0, 0)'
            )
        )
    )
    data = [trace0, trace1]
    #data = [trace0]

    layout = dict(title = 'Sentiment(using '+libraryName+ ') scatter plot for ' + nutrient + ' ' + '(' + year +')',
                  yaxis = dict(title = 'Polarity',zeroline = False),
                  xaxis = dict(title = 'Statement #(not in original order)',zeroline = False)
                 )

    fig = dict(data=data, layout=layout)
    return fig;

In [81]:
requiredNutrient = 'Vitamin'
requiredYear = '2010'

iplot(plotPolarityScatterPlot(nutrient = requiredNutrient,nutrientStatementPolarityList = getNutrientRelevantStatementsPolarityListUsingTextBlob(year = requiredYear,nutrientName = requiredNutrient ), year = requiredYear,libraryName='TextBlob'), filename='sugar-polarity-scatter-2015')

In [82]:
requiredNutrient = 'Vitamin'
requiredYear = '2010'

iplot(plotPolarityScatterPlot(nutrient = requiredNutrient,nutrientStatementPolarityList = getNutrientRelevantStatementsPolarityListUsingVader(year = requiredYear,nutrientName = requiredNutrient ), year = requiredYear,libraryName='Vader'), filename='sugar-polarity-scatter-2015')

In [83]:
def getNutrientPolarityTrendLineUsingTextBlob (nutrient):
    import numpy as np
    yearsList = np.arange(1980,2020,5)
    polarityMeansAcrossYearsList = []
    for i in yearsList:
        polarityMeansAcrossYearsList.append(normalizedMean(getNutrientRelevantStatementsPolarityListUsingTextBlob(year=str(i),nutrientName = nutrient)))
    data = [go.Scatter(
          x=np.arange(1980,2020,5),
          y=polarityMeansAcrossYearsList)]
    layout = dict(
         title = "Trends(using TextBlob) in the sentiment of " + nutrient + '(s) ' + '(1980-2015)' ,
         yaxis = dict(title = 'Average Polarity(Normalized)'),
         xaxis = dict(title = 'Year')
    )

    fig = dict(data=data, layout=layout)
    return fig;

In [84]:
def getNutrientRelevantStatementsPolarityDictionaryUsingTextBlob(year, nutrientName): 
    with open(year + '.txt', 'r') as myfile:
        text = myfile.read()
    text1 =text.replace('\n________________\n','\n')
    text2 = re.sub(r'\n([a-z]+)',r' \1', text1)
    list1 = [sentence for sentence in text2.split('\n')]
    list2 = []
    for i in list1:
        list2.append(i.split('.'))
        
    
    
    finalList = []
    
    for i in list2:
            for j in i:
                finalList.append(j)
    
    nutrientStatementList = []
    
    for i in finalList:
        if nutrientName.lower() in i:
            nutrientStatementList.append(i)
    
    
    dict = {}
    
    for i in nutrientStatementList:
            dict[i] = TextBlob(i).sentiment.polarity
        
        
    return dict; 




In [85]:
def getNutrientRelevantStatementsPolarityDictionaryUsingVader(year, nutrientName): 
    with open(year + '.txt', 'r') as myfile:
        text = myfile.read()
    text1 =text.replace('\n________________\n','\n')
    text2 = re.sub(r'\n([a-z]+)',r' \1', text1)
    list1 = [sentence for sentence in text2.split('\n')]
    list2 = []
    for i in list1:
        list2.append(i.split('.'))
        
    
    
    finalList = []
    
    for i in list2:
            for j in i:
                finalList.append(j)
    
    nutrientStatementList = []
    
    for i in finalList:
        if nutrientName.lower() in i:
            nutrientStatementList.append(i)
    
    
    dict = {}
    
    for i in nutrientStatementList:
            dict[i] = (analyzer.polarity_scores(i))['compound']
        
        
    return dict; 

In [86]:
for statement, polarity in getNutrientRelevantStatementsPolarityDictionaryUsingTextBlob(year = '2015', nutrientName = 'Carbohydrate').items():
    print('{} ==> {}'.format(statement, polarity))

NOTE: The total eating pattern should not exceed Dietary Guidelines limits for intake of calories from added sugars and saturated fats and alcohol and should be within the Acceptable Macronutrient Distribution Ranges for calories from protein, carbohydrate, and total fats ==> 0.0
 On average, carbohydrates and protein contain 4 calories per gram, fats contain 9 calories per gram, and alcohol has 7 calories per gram ==> -0.15
 Strong evidence from mostly prospective cohort studies but also randomized controlled trials has shown that eating patterns that patterns can accommodate other nutrient- dense foods with small amounts of added sugars, such as whole-grain breakfast cereals or fat-free yogurt, as long as calories from added sugars do not exceed 10 percent per day, total carbohydrate intake remains within the AMDR, and total calorie intake remains within limits ==> 0.06354166666666666
 with carbohydrates reduces blood levels ==> 0.0
Dietary Guidelines to limit consumption saturated f

In [87]:
def getNutrientRelevantStatementsPolarityListUsingVader(year, nutrientName): 
    with open(year + '.txt', 'r') as myfile:
        text = myfile.read()
    text1 =text.replace('\n________________\n','\n')
    text2 = re.sub(r'\n([a-z]+)',r' \1', text1)
    list1 = [sentence for sentence in text2.split('\n')]
    list2 = []
    for i in list1:
        list2.append(i.split('.'))
    
    finalList = []
    
    for i in list2:
            for j in i:
                finalList.append(j)
    
    nutrientStatementList = []
    
    for i in finalList:
        if nutrientName.lower() in i:
            nutrientStatementList.append(i)
    
    
    statementsPolarityList = []
    
    for i in nutrientStatementList:
            statementsPolarityList.append((analyzer.polarity_scores(i))['compound'])
        
        
    return statementsPolarityList; 




In [88]:
def getNutrientPolarityTrendLineUsingVader (nutrient):
    import numpy as np
    yearsList = np.arange(1980,2020,5)
    polarityMeansAcrossYearsList = []
    for i in yearsList:
        polarityMeansAcrossYearsList.append(normalizedMean(getNutrientRelevantStatementsPolarityListUsingVader(year=str(i),nutrientName = nutrient)))
    data = [go.Scatter(
          x=np.arange(1980,2020,5),
          y=polarityMeansAcrossYearsList)]
    layout = dict(
         title = "Trends(using Vader) in the sentiment of " + nutrient + '(s) ' + '(1980-2015)' ,
         yaxis = dict(title = 'Average Polarity(Normalized)'),
         xaxis = dict(title = 'Year')
    )

    fig = dict(data=data, layout=layout)
    return fig;

In [89]:
import plotly.plotly as py
init_notebook_mode(connected=True)

nutrientName = 'Carbohydrate'

iplot(getNutrientPolarityTrendLineUsingVader(nutrientName)) 

In [90]:
import plotly.plotly as py
init_notebook_mode(connected=True)

nutrientName = 'Carbohydrate'

iplot(getNutrientPolarityTrendLineUsingTextBlob(nutrientName)) 

In [91]:
for statement, polarity in getNutrientRelevantStatementsPolarityDictionaryUsingVader(year = '2015', nutrientName = 'Carbohydrate').items():
    print('{} ==> {}'.format(statement, polarity))

NOTE: The total eating pattern should not exceed Dietary Guidelines limits for intake of calories from added sugars and saturated fats and alcohol and should be within the Acceptable Macronutrient Distribution Ranges for calories from protein, carbohydrate, and total fats ==> 0.3182
 On average, carbohydrates and protein contain 4 calories per gram, fats contain 9 calories per gram, and alcohol has 7 calories per gram ==> 0.0
 Strong evidence from mostly prospective cohort studies but also randomized controlled trials has shown that eating patterns that patterns can accommodate other nutrient- dense foods with small amounts of added sugars, such as whole-grain breakfast cereals or fat-free yogurt, as long as calories from added sugars do not exceed 10 percent per day, total carbohydrate intake remains within the AMDR, and total calorie intake remains within limits ==> 0.2846
 with carbohydrates reduces blood levels ==> 0.0
Dietary Guidelines to limit consumption saturated fats with car